In [119]:
import findspark
import os

findspark.init(os.environ.get('SPARK_HOME'))

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import pyspark.sql.functions as functions
from decimal import Decimal
from pyspark.sql import DataFrame

import numpy as np
import pandas as pd
import sys
from argparse import ArgumentParser
import functools

In [120]:
spark = SparkSession.builder.appName("GastoMinisterio").getOrCreate()

In [121]:
#def processarArquivo(caminhoArquivo) :
    

In [122]:
caminhoArquivo = "hdfs://localhost:9000/user/ubuntu/2019_Viagem.csv"

In [123]:
def processarArquivo(caminhoArquivo) :
    print(caminhoArquivo)
    to_value = lambda v : Decimal(v.replace(",", "."))
    udf_to_value = functions.udf(to_value)
    df = spark.read.format("csv").option("header", True).option("delimiter", ";").option('encoding', 'windows-1252').csv(caminhoArquivo)   
    df.printSchema()
    df2 = df.withColumn("value", udf_to_value(df["Valor passagens"]))
    orgaos = df2.select("Nome do órgão superior").distinct()
    agrupamento = df2.groupBy("Nome do órgão superior").agg(functions.max("value"), functions.sum("value"), functions.count("value"), functions.avg("value"))
    agrupamento.show(orgaos.count(), truncate = False)
    return agrupamento

In [124]:
processarArquivo(caminhoArquivo)

hdfs://maycon:9000/user/maycon/2019_Viagem.csv
root
 |-- Identificador do processo de viagem: string (nullable = true)
 |-- Situação: string (nullable = true)
 |-- Código do órgão superior: string (nullable = true)
 |-- Nome do órgão superior: string (nullable = true)
 |-- Código órgão solicitante: string (nullable = true)
 |-- Nome órgão solicitante: string (nullable = true)
 |-- CPF viajante: string (nullable = true)
 |-- Nome: string (nullable = true)
 |-- Cargo: string (nullable = true)
 |-- Período - Data de início: string (nullable = true)
 |-- Período - Data de fim: string (nullable = true)
 |-- Destinos: string (nullable = true)
 |-- Motivo: string (nullable = true)
 |-- Valor diárias: string (nullable = true)
 |-- Valor passagens: string (nullable = true)
 |-- Valor outros gastos: string (nullable = true)

+-----------------------------------------------------------+----------+--------------------+------------+------------------+
|Nome do órgão superior                        

DataFrame[Nome do órgão superior: string, max(value): string, sum(value): double, count(value): bigint, avg(value): double]

In [125]:
argumentos = []
for i in sys.argv :
    if(i.endswith(".csv")): 
        argumentos.append(i);
lista = []
if len(argumentos) == 0 : 
    lista.append(processarArquivo(caminhoArquivo))
else :
    for i in argumentos :
        lista.append(processarArquivo(i)) 

hdfs://maycon:9000/user/maycon/2019_Viagem.csv
root
 |-- Identificador do processo de viagem: string (nullable = true)
 |-- Situação: string (nullable = true)
 |-- Código do órgão superior: string (nullable = true)
 |-- Nome do órgão superior: string (nullable = true)
 |-- Código órgão solicitante: string (nullable = true)
 |-- Nome órgão solicitante: string (nullable = true)
 |-- CPF viajante: string (nullable = true)
 |-- Nome: string (nullable = true)
 |-- Cargo: string (nullable = true)
 |-- Período - Data de início: string (nullable = true)
 |-- Período - Data de fim: string (nullable = true)
 |-- Destinos: string (nullable = true)
 |-- Motivo: string (nullable = true)
 |-- Valor diárias: string (nullable = true)
 |-- Valor passagens: string (nullable = true)
 |-- Valor outros gastos: string (nullable = true)

+-----------------------------------------------------------+----------+--------------------+------------+------------------+
|Nome do órgão superior                        

In [126]:
dfUnido = lista[0]
i = 0
for data in lista : 
    if(i < (len(lista)-1)):
        dfUnido = dfUnido.union(lista[i + 1])
        i = i + 1
    

In [138]:
dfUnido.show(100)

+----------------------+----------+--------------------+------------+------------------+
|Nome do órgão superior|max(value)|          sum(value)|count(value)|        avg(value)|
+----------------------+----------+--------------------+------------+------------------+
|  Ministério do Mei...|    989.92|   4754103.779999999|       11766| 404.0543753187149|
|  Ministério da Agr...|    998.83|   6569691.700000001|       18862|348.30302725055674|
|  Advocacia-Geral d...|    994.57|   978350.1899999995|        3696|264.70513798701285|
|  Ministério das Ci...|      0.00|                 0.0|           1|               0.0|
|  Ministério da Cul...|   3494.56|              5256.2|           2|            2628.1|
|  Ministério da Cid...|    998.58|          1823496.37|        1923| 948.2560426417057|
|  Presidência da Re...|    999.91|   6874017.199999995|        5309|1294.7856846863806|
|  Ministério da Ciê...|    999.24|   5865138.879999997|        4647|1262.1344695502469|
|  Ministério da Defe

In [168]:
dfUnido.write.mode('overwrite').csv("hdfs://localhost:9000/user/ubuntu/agrupamento.csv")

Py4JJavaError: An error occurred while calling o2431.csv.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:198)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:159)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:127)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.sc